# YOLO, so Save Water – YOLOv5 Nano edition

This notebook will assess the performance of YOLOv5 for detection of taps in pictures, differentiating between taps with and without running water.

This notebook is heavily based on the notebook from [Ultralytics's tutorial](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data).

## Step 0. Setup

We will use Roboflow to manage conversion of the dataset to a format that is appropriate for YOLOv5. Roboflow provides a nice library of bindings for Python to easily extract the dataset.

In [ ]:
!pip install roboflow --quiet

Now we will clone the YOLOv5 repository and install its dependencies:

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt --quiet

import torch
import os
from IPython.display import Image, clear_output

## Step 1. Downloading the custom dataset

Here we will fetch the custom dataset. It is available in my account in Roboflow.

**Disclaimer**: the API key in the code below is revoked, the data cannot be accessed with it.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="3FpNhe5OoFPSZw7qdSq7")
project = rf.workspace().project("yolo--so-save-water")
dataset = project.version(2).download("yolov5")

## Step 2. Training

In [ ]:
!python train.py --img 640 --batch 16 --epochs 1000 --data {dataset.location}/data.yaml --weights yolov5n.pt --cache

# Step 3. Inference

Let's now test how our trained model performs on unseen data from the test dataset.

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.6 --source {dataset.location}/test/images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'):
    display(Image(filename=imageName))
    print('\n')

As we can see from the images above, the model performance is not good. The guesses aren't confident enough, so with the 0.6 threshold, many of them simply do not come through.

# Step 4. Evaluating metrics

Metrics are plotted by TensorBoard. It seems that mAP isn't too bad, reaching around 0.95 in peaks. However, that doesn't seem to transfer to the model's performance.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs